Watch DM progress

In [1]:
import os
import time
import numpy as np
import xarray as xr
result_dir = '/usr/scratch4/sima9999/work/modal_uq/uq_modal_beam/'
path = os.path.join(result_dir, 'samples')

In [1]:

in_ds = xr.load_dataset(path + '/uq_modal_beam.nc')
out_ds = xr.load_dataset(path + '/uq_modal_beam_out.nc')
np.isnan(out_ds._runtimes.data)[0][0]

In [ ]:
for i in range(3938021,7599653): # 3938021 is the first invalid # 3625888 is broken, is not broken?
    if not i%250:
        print('.', end='', flush=True)
    jid = in_ds.ids[i].item()
    res_dir = os.path.join(path,jid,f'{jid}_mechanical.npz')
    arr = np.load(res_dir, allow_pickle=True)
    assert arr['self.struct_parms'].item()['dy_tmd'] == in_ds['dD'][i].item()

In [34]:
display(i)

3938021

In [38]:
jid = in_ds.ids[i].item()
res_dir = os.path.join(path,jid,f'{jid}_mechanical.npz')
print(time.ctime(os.stat(res_dir).st_mtime))
print(time.ctime(os.stat(res_dir).st_ctime))

Mon Nov  7 09:21:49 2022
Mon Nov  7 09:21:49 2022


- 9,999,693 = 729 * 13,717 # total samples
- 3,930,727 / 729 = 5,391.9 # last datamanager sample with runtime not nan -> start_ale=5390 is the last good sample when plotting in estimate_imp
- 6,581,412 = 729 * 9,028 # last datamanager sample with results not nan
- 3,938,021 / 729 = 5,401.9 # last sample, that progress watcher marks valid, probably the last 7294 samples were not saved into db -> it is bad when plotting in estimate_imp

In [50]:
for ret_name in ['damp_freqs','zetas','frf']:
    
    if ret_name == 'frf':
        # continue
        inds = range(1025*2)
    else:
        # continue
        inds = range(14)
    all_times = []
    all_dirs = []
    all_progr = []
    for ind in inds:
        if ret_name == 'frf':
            ret_ind = {'frequencies':ind//2, 'space':ind%2}
            if not ind%2: continue
        else:
            ret_ind = {'modes':ind}
            
        ret_dir = f'{ret_name}-{".".join(str(e) for e in ret_ind.values())}'
        fname = os.path.join(result_dir, 'estimations', f'{ret_dir}/polyuq_imp.npz')
        # print(ret_dir)
        if os.path.exists(fname):
            all_times.append(os.stat(fname).st_mtime)
            all_dirs.append(ret_dir)
            try:
                intp_err = np.load(fname)['self.intp_errors']
            except:
                all_progr.append(0)
                continue
            nan_ind = np.where(np.isnan(intp_err))[0]
            if len(nan_ind):
                all_progr.append(np.min(nan_ind))
            else:
                all_progr.append(len(intp_err))
        else:
            all_progr.append(0)
            all_dirs.append(ret_dir)
            all_times.append(0)
    
    display(f'finished: {np.sum(np.array(all_progr)==5390)}, not finished {np.sum(np.array(all_progr)!=5390)}')
    for ret_time,ret_dir, progr in sorted(zip(all_times, all_dirs, all_progr)):
        # if progr == 5390: continue
        # if 1675933971.0 <= ret_time <= 1675934235.0 and progr==0: continue
        print(f'{ret_dir} @ {progr} on {time.ctime(ret_time)} {ret_time}')

'finished: 14, not finished 0'

damp_freqs-6 @ 5390 on Mon Feb  6 20:03:13 2023 1675710193.0
damp_freqs-8 @ 5390 on Mon Feb  6 20:07:40 2023 1675710460.0
damp_freqs-12 @ 5390 on Mon Feb  6 20:11:20 2023 1675710680.0
damp_freqs-1 @ 5390 on Mon Feb  6 20:13:24 2023 1675710804.0
damp_freqs-0 @ 5390 on Mon Feb  6 20:16:07 2023 1675710967.0
damp_freqs-10 @ 5390 on Mon Feb  6 20:17:12 2023 1675711032.0
damp_freqs-11 @ 5390 on Mon Feb  6 20:24:43 2023 1675711483.0
damp_freqs-2 @ 5390 on Mon Feb  6 20:29:20 2023 1675711760.0
damp_freqs-9 @ 5390 on Mon Feb  6 20:33:11 2023 1675711991.0
damp_freqs-4 @ 5390 on Mon Feb  6 20:34:15 2023 1675712055.0
damp_freqs-7 @ 5390 on Mon Feb  6 20:44:08 2023 1675712648.0
damp_freqs-5 @ 5390 on Mon Feb  6 20:57:54 2023 1675713474.0
damp_freqs-3 @ 5390 on Mon Feb  6 21:07:07 2023 1675714027.0
damp_freqs-13 @ 5390 on Tue Feb  7 05:29:10 2023 1675744150.0


'finished: 14, not finished 0'

zetas-4 @ 5390 on Mon Feb  6 20:47:46 2023 1675712866.0
zetas-0 @ 5390 on Mon Feb  6 20:56:28 2023 1675713388.0
zetas-7 @ 5390 on Mon Feb  6 21:07:23 2023 1675714043.0
zetas-5 @ 5390 on Mon Feb  6 21:09:43 2023 1675714183.0
zetas-1 @ 5390 on Mon Feb  6 21:13:26 2023 1675714406.0
zetas-2 @ 5390 on Mon Feb  6 21:15:26 2023 1675714526.0
zetas-6 @ 5390 on Mon Feb  6 21:22:01 2023 1675714921.0
zetas-3 @ 5390 on Mon Feb  6 21:30:21 2023 1675715421.0
zetas-12 @ 5390 on Mon Feb  6 21:34:08 2023 1675715648.0
zetas-13 @ 5390 on Mon Feb  6 21:34:37 2023 1675715677.0
zetas-10 @ 5390 on Mon Feb  6 21:40:55 2023 1675716055.0
zetas-9 @ 5390 on Mon Feb  6 21:41:12 2023 1675716072.0
zetas-8 @ 5390 on Mon Feb  6 21:41:55 2023 1675716115.0
zetas-11 @ 5390 on Mon Feb  6 21:43:35 2023 1675716215.0


'finished: 708, not finished 317'

frf-1000.1 @ 0 on Thu Jan  1 01:00:00 1970 0
frf-1001.1 @ 0 on Thu Jan  1 01:00:00 1970 0
frf-1002.1 @ 0 on Thu Jan  1 01:00:00 1970 0
frf-1003.1 @ 0 on Thu Jan  1 01:00:00 1970 0
frf-1004.1 @ 0 on Thu Jan  1 01:00:00 1970 0
frf-1005.1 @ 0 on Thu Jan  1 01:00:00 1970 0
frf-1006.1 @ 0 on Thu Jan  1 01:00:00 1970 0
frf-1007.1 @ 0 on Thu Jan  1 01:00:00 1970 0
frf-1008.1 @ 0 on Thu Jan  1 01:00:00 1970 0
frf-1009.1 @ 0 on Thu Jan  1 01:00:00 1970 0
frf-1010.1 @ 0 on Thu Jan  1 01:00:00 1970 0
frf-1011.1 @ 0 on Thu Jan  1 01:00:00 1970 0
frf-1012.1 @ 0 on Thu Jan  1 01:00:00 1970 0
frf-1013.1 @ 0 on Thu Jan  1 01:00:00 1970 0
frf-1014.1 @ 0 on Thu Jan  1 01:00:00 1970 0
frf-1015.1 @ 0 on Thu Jan  1 01:00:00 1970 0
frf-1016.1 @ 0 on Thu Jan  1 01:00:00 1970 0
frf-1017.1 @ 0 on Thu Jan  1 01:00:00 1970 0
frf-1018.1 @ 0 on Thu Jan  1 01:00:00 1970 0
frf-1019.1 @ 0 on Thu Jan  1 01:00:00 1970 0
frf-1020.1 @ 0 on Thu Jan  1 01:00:00 1970 0
frf-1021.1 @ 0 on Thu Jan  1 01:00:00 1970 0
frf-1022.1

In [67]:
ret_dir = 'frf-585.0'
fname = os.path.join(result_dir, 'estimations', f'{ret_dir}/polyuq_imp.npz')
os.stat(fname).st_mtime > 1675778682.0

True

In [ ]:
ret_name = 'frf'
for freq_ind in range(1025):
    ret_ind = {'frequencies':freq_ind, 'space':0}
    ret_dir = f'{ret_name}-{".".join(str(e) for e in ret_ind.values())}'
    fname = os.path.join(result_dir, 'estimations', f'{ret_dir}/polyuq_imp.npz')
    
    if os.path.exists(fname):
        if os.stat(fname).st_mtime < 1675932359.0:
            os.rename(fname, fname + '.bak')
        else:
            
            print(time.ctime(os.stat(fname).st_mtime))
    else:
        print(ret_dir)
        f = open(fname,'wb')
        f.close()

In [35]:
with open('/usr/scratch4/sima9999/output.log') as f:
    _line=''
    for line in f:
        if line==_line: continue
        
        if 'makalu' in line or 'frf-515' in line or 'frf-525' in line or 'frf-536' in line or 'frf-537' in line or 'frf-538' in line or 'frf-539' in line or 'frf-540' in line or 'frf-541' in line or 'frf-542' in line:
            print(line)
        _line = line
    




makalu110.rz.tu-ilmenau.de

/usr/tmp/ray/session_latest/logs/worker-03b65e39fa97ab0b8ec613499b5a04695635df895695a39b1838347d-02000000-2830.out:INFO:uncertainty.polymorphic_uncertainty:Now loading previous results from  /usr/scratch4/sima9999/work/modal_uq/uq_modal_beam/estimations/frf-515.0/polyuq_imp.npz

/usr/tmp/ray/session_latest/logs/worker-03b65e39fa97ab0b8ec613499b5a04695635df895695a39b1838347d-02000000-2830.out:INFO:uncertainty.polymorphic_uncertainty:Saving state of PolyUQ to /usr/scratch4/sima9999/work/modal_uq/uq_modal_beam/estimations/frf-515.0/polyuq_imp.npz. Make sure to store your variable definitions script externally...

/usr/tmp/ray/session_latest/logs/worker-2b76a707f06ce590646cde609577cfdd3d87e8ac0c8caad8b860f4d5-02000000-3035.out:INFO:uncertainty.polymorphic_uncertainty:Now loading previous results from  /usr/scratch4/sima9999/work/modal_uq/uq_modal_beam/estimations/frf-525.0/polyuq_imp.npz

/usr/tmp/ray/session_latest/logs/worker-2b76a707f06ce590646cde609577cfdd3d